In [68]:
import numpy as np
import numpy.random as rd
import sys
import matplotlib.pyplot as plt

dataset_num = 100
variance = 0.5
wtflag = 0
learning_rate = 0.2
file_exist = False

epochs = 300

# making dataset


def making_data_set():
    d_set = np.empty(1)
    if file_exist:
        tmp = np.load(sys.argv[1])
        d_set = np.append(d_set, tmp)
    else:
        prob = rd.randint(0, 8, dataset_num)
        for i in prob:
            b0 = int(i/2)
            b1 = int(b0/2)
            b2 = int(b1/2)
            
            x = rd.normal(b0*2 - 1, variance)
            y = rd.normal(b1*2 -1, variance)
            z = rd.normal(b2*2-1, variance)
            
            result = b0 + b1 + b2
            
            rb0 = int(result/2)
            rb1 = int(rb0/2)
            
            '''x = rd.randint(0, 2)
            y = rd.randint(0, 2)
            rst = x | y'''
            d_set = np.append(d_set, [[x, y, z, rb0, rb1]])
        if wtflag == 1:
            np.save('dataset.npy', d_set)
    d_set = np.delete(d_set, 0, axis=0)
    d_set = d_set.reshape(-1, 5)
    return d_set
t_data = making_data_set()
t_x_data = t_data[:, 0:-2]
t_y_data = t_data[:, -2:].reshape(-1, 2)

test_data = making_data_set()
test_x_data = t_data[:, 0:-2]
test_y_data = t_data[:, -2:].reshape(-1, 2)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class layer_class:
    w = np.array([]); bias = 'False'; b = np.array([]);  dataflow = np.array([]);  delta = np.array([]);  y_ = 0
    def __init__(self, w, bias='False'):
        self.w = w
        if bias == 'True':
            self.bias = 'True'
            self.b = np.zeros((1, w.shape[1]))
        else:
            self.bias = 'False'






In [ ]:
class NN:
    train_set_x, train_set_y, test_set_x, test_set_y, learning_rate,  activate = np.array([]),np.array([]),np.array([]),np.array([]), 0, 'sigmoid'
    layer, train_loss, test_loss, accuracy = [], [], [], []
    feature = []
    
    #추적
    weight_hist = []
    delta_hist = []
    bias_hist = []
    def __init__(self, train_set_x, train_set_y, test_set_x, test_set_y, feature, learning_rate=0.01, activate='sigmoid'):
        self.learning_rate = learning_rate
        self.activate = activate
        self.train_set_x = train_set_x
        self.train_set_y = train_set_y
        self.test_set_x = test_set_x
        self.test_set_y = test_set_y
        self.feature = feature
        
        # w initialization sigmoid use xavier initialization  Relu using He initialization
        for i, f in enumerate(self.feature):
            if i == (len(self.feature)-1):
                break
            w = 0
            if f[2] is 'sigmoid':
                w = rd.randn(f[0], int(self.feature[i + 1][0]))* np.sqrt(1/(f[0]+self.feature[i + 1][0]))
            elif f[2] is 'relu':
                w = rd.randn(f[0], int(self.feature[i + 1][0])) * np.sqrt(2/(f[0]+self.feature[i + 1][0])) 
            self.layer.append(layer_class(w, f[1]))

    def sigmoid(self, a):
        return 1/(1 + np.exp(-a)) 
    def relu(self, a):
        a = a if a > 0 else 0
        return a
    def dsigmoid(self, a):
        return a * (1 - a)
    def drelu(self, a):
        if a > 0:
            return 1
        else:
            return 0

    # forwarding
    def pri(self):
        for i in self.layer:
            print(i.w)
        print("\n", self.layer[0].dataflow)

    def forwarding(self):
        ftn = 0
        if self.activate == 'sigmoid':
            ftn = self.sigmoid
        elif self.activate == 'relu':
            ftn = self.relu

        for i, k in enumerate(self.layer):
            a = np.array([])
            if i is 0:
                a = np.append(a, np.dot(self.train_set_x, k.w))
            else:
                a = np.append(a, np.dot(self.layer[i-1].dataflow, k.w))
            
            a = a.reshape(self.train_set_x.shape[0], k.w.shape[1])
            
            if k.bias == 'True':
                #print(f"ones : {type(np.ones(1))}  bias : {type(k.b)}")
                a = np.multiply(np.ones((int(self.train_set_x.shape[0]), int(k.w.shape[1]))) , k.b) + a
                #print(a)
                
            k.dataflow = ftn(a)
            
        #print(f"predic output : {self.layer[-1].dataflow.shape}\n data output : {self.train_set_y.shape}")
        self.train_loss = np.append(self.train_loss, np.average((self.layer[-1].dataflow - self.train_set_y) ** 2))

        for i, k in enumerate(self.layer):
            a = np.array([])
            if i is 0:
                a = np.append(a, np.dot(self.test_set_x, k.w))
            else:
                a = np.append(a, np.dot(self.layer[i-1].dataflow, k.w))
            a = a.reshape(self.test_set_x.shape[0], k.w.shape[1])
            k.dataflow = ftn(a)
        self.test_loss = np.append(self.test_loss, np.average((self.layer[-1].dataflow - self.train_set_y) ** 2))


    def backpropagation(self):
        dftn = 0
        if self.activate == 'sigmoid':
            dftn = self.dsigmoid
        elif self.activate == 'relu':
            dftn = self.drelu


        for i, L in reversed(list(enumerate(self.layer))):
            '''summation {}{j} wij delatj 교재 slide 12p'''

            # fully connected layer
            if i is (len(self.layer)-1):
                L.delta = np.multiply((self.train_set_y - L.dataflow), dftn(L.dataflow))
                #print(f"$$$$$$$$$i :{i}, L.dataflow : {L.dataflow.shape} L.delta : {L.delta.shape}, L.w : {L.w.shape}")
            #else
            else:
                back_L = self.layer[i+1]
                L.delta = np.multiply(dftn(L.dataflow), np.dot(back_L.delta, back_L.w.T))
                # print(f"L.delta 구하는 과정\n\ni :{i}, L.dataflow : {L.dataflow.shape} back_L.delta : {back_L.delta.shape}, back_L.w : {back_L.w.shape}, L.w. : {L.w.shape}\n 구하기 end \n\n")

            # i가 0 이면 input data가 train_set
            if i is 0 :
                input = self.train_set_x
            else:
                input = self.layer[i - 1].dataflow

            '''x_k * delta -> upgrade할 weight '''
            upgrade_w = np.zeros(1)
            #outer로 한꺼번에 추가후에, reshape로 정렬
            for k in range(L.delta.shape[0]):
                upgrade_w = np.append(upgrade_w, np.outer(input[k], L.delta[k]))
                #print(f" for {i} layer **********************\ndelta : \n{L.delta[k]}\n input : \n{input[k]}\n result : \n{np.outer(input[k], L.delta[k])}")
            upgrade_w = np.delete(upgrade_w, 0, axis=0)
            upgrade_w = upgrade_w.reshape(-1, L.w.shape[0], L.w.shape[1])
            
            self.delta_hist.append(L.delta.reshape(-1))
            
            if i == 0:
                self.weight_hist.append(L.w.reshape(-1))
            if L.bias is "True":
                self.bias_hist.append(L.b.reshape(-1))
            
            """weight update"""
            if L.bias == 'True':
                L.b = L.b + np.average(L.delta, axis=0)
            
            """update weight"""
            L.w = L.w + np.average(upgrade_w, axis=0) * self.learning_rate
            
            

    def training(self, epochs):
        for i in range(epochs):
            self.forwarding()
            self.backpropagation()
            
            cnt = np.array(np.abs(self.test_set_y  - np.add(self.layer[-1].dataflow, -0.2)) < 0.4 )
            #print(f"ny_data : \n{test_y_data}\n\n y^ : \n{self.layer[-1].dataflow}")
            correct = np.average(cnt) * 100
            self.accuracy.append(correct)
            #print(f"bias : {self.layer[0].b}")
            print(f"epochs : {i}\t train_loss :{self.train_loss[i]:.5}\t test_loss :{self.test_loss[i]:.5}, accuracy :{correct}")

    def plot(self):

        plt.figure()
        plt.plot(self.train_loss, 'b')
        plt.plot(self.test_loss, 'r')
        plt.show()

        plt.figure()
        plt.plot(self.accuracy)
        plt.show()

        plt.figure()
        plt.plot(self.weight_hist)
        plt.plot(self.delta_hist)
        plt.show()
        
        plt.figure()
        plt.plot(self.bias_hist)
        plt.show()
        


test = NN(t_x_data, t_y_data, test_x_data, test_y_data, [[3, 'True', 'sigmoid'],[100, 'True', 'sigmoid'], [100, 'True', 'sigmoid'],[2, 'False', 'sigmoid']], learning_rate=learning_rate)

test.training(epochs)
test.plot()

epochs : 0	 train_loss :0.46158	 test_loss :0.46158, accuracy :65.5
epochs : 1	 train_loss :0.43965	 test_loss :0.44238, accuracy :65.5
epochs : 2	 train_loss :0.43114	 test_loss :0.43393, accuracy :65.5
epochs : 3	 train_loss :0.42788	 test_loss :0.43005, accuracy :38.5
epochs : 4	 train_loss :0.42658	 test_loss :0.4281, accuracy :38.5
epochs : 5	 train_loss :0.42604	 test_loss :0.42701, accuracy :38.5
epochs : 6	 train_loss :0.4258	 test_loss :0.42633, accuracy :38.5
epochs : 7	 train_loss :0.42567	 test_loss :0.42587, accuracy :38.5
epochs : 8	 train_loss :0.42557	 test_loss :0.42551, accuracy :38.5
epochs : 9	 train_loss :0.42547	 test_loss :0.42521, accuracy :38.5
epochs : 10	 train_loss :0.42537	 test_loss :0.42495, accuracy :38.5
epochs : 11	 train_loss :0.42524	 test_loss :0.42471, accuracy :38.5
epochs : 12	 train_loss :0.42511	 test_loss :0.42448, accuracy :38.5
epochs : 13	 train_loss :0.42496	 test_loss :0.42426, accuracy :38.5
epochs : 14	 train_loss :0.42479	 test_loss :0